# code_utils

> Stata-related helper functions with no Jupyter or pystata dependence
- order: 4

These are mostly intended for use in the `noecho` module.

In [ ]:
#| default_exp code_utils
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import re

In [ ]:
#| export
from pygments import lexers
from pygments.token import Comment

In [ ]:
from fastcore.test import test_eq
from textwrap import dedent

## Handling Stata comments and `#delimit;`

In [ ]:
#| export

# Detect comments spanning multiple lines
comment_regex = re.compile(r'(((?: |\t)\/\/\/)(.)*(\n|\r)|(\/\*)(.|\s)*?(\*\/))')

def _remove_multi_line_comments(code):
    return comment_regex.sub(' ',code)

In [ ]:
show_doc(_remove_multi_line_comments)

---

[source](https://github.com/hugetim/nbstata/blob/main/nbstata/code_utils.py#L14){target="_blank" style="float:right; font-size:smaller"}

### _remove_multi_line_comments

>      _remove_multi_line_comments (code)

In [ ]:
test_eq(
    _remove_multi_line_comments(dedent("""\
        disp ///
        1""")),
    "disp 1")

Correctly ignores "///" when not preceded by a space:

In [ ]:
test_eq(
    _remove_multi_line_comments(dedent("""\
        disp///
        1
        """)),
    dedent("""\
        disp///
        1
        """)
)

In [ ]:
test_eq(
    _remove_multi_line_comments(dedent("""\
        /*
        blah
        blah
        */
        list var
        """)),
    """\
 
list var
"""
)

In [ ]:
#| hide
#| export
stata_lexer = lexers.get_lexer_by_name('stata')

def _lex_tokens(code):
    return stata_lexer.get_tokens_unprocessed(code)

#| hide
* [https://pygments.org/docs/api/#pygments.lexer.Lexer.get_tokens_unprocessed](https://pygments.org/docs/api/#pygments.lexer.Lexer.get_tokens_unprocessed)
* [https://github.com/pygments/pygments/blob/master/pygments/lexers/stata.py](https://github.com/pygments/pygments/blob/master/pygments/lexers/stata.py)
* [https://github.com/pygments/pygments/blob/master/pygments/token.py](https://github.com/pygments/pygments/blob/master/pygments/token.py)

In [ ]:
#| export
def remove_comments(code):
    return "".join(token[2] for token in _lex_tokens(code) if token[1] not in Comment)

In [ ]:
list(_lex_tokens('tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19", "AG4B to AGB", "AG5B to AGC")  //"E20 to B20", ", sort'))

[(0, Token.Keyword, 'tab'),
 (3, Token.Text, ' '),
 (4, Token.Text, 's'),
 (5, Token.Text, 'i'),
 (6, Token.Text, 'z'),
 (7, Token.Text, 'e'),
 (8, Token.Keyword, ' if'),
 (11, Token.Text, ' '),
 (12, Token.Name.Function, 'inlist'),
 (18, Token.Text, '('),
 (19, Token.Text, 'r'),
 (20, Token.Text, 't'),
 (21, Token.Text, '_'),
 (22, Token.Text, 'f'),
 (23, Token.Text, 'r'),
 (24, Token.Text, 'o'),
 (25, Token.Text, 'm'),
 (26, Token.Text, '_'),
 (27, Token.Text, 't'),
 (28, Token.Text, 'o'),
 (29, Token.Text, ','),
 (30, Token.Text, ' '),
 (31, Token.Literal.String, '"'),
 (32, Token.Literal.String, 'A'),
 (33, Token.Literal.String, '1'),
 (34, Token.Literal.String, '0'),
 (35, Token.Literal.String, 'T'),
 (36, Token.Literal.String, 'O'),
 (37, Token.Literal.String, 'U'),
 (38, Token.Literal.String, ' '),
 (39, Token.Literal.String, 't'),
 (40, Token.Literal.String, 'o'),
 (41, Token.Literal.String, ' '),
 (42, Token.Literal.String, 'B'),
 (43, Token.Literal.String, '1'),
 (44, Token.L

In [ ]:
remove_comments('tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  //"E20 to B20", ", sort')

'tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  '

In [ ]:
test_eq(remove_comments('*tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  //"E20 to B20", ", sort'), 
        '')

In [ ]:
test_eq(remove_comments('tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  /*"E20 to B20", ", sort'),
        'tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  ')

In [ ]:
test_eq(remove_comments('tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  /*"E20 to B20", "*/, sort'),
        'tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  , sort')

In [ ]:
test_eq(remove_comments('tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")  ///"E20 to B20", "*/ \n , sort'),
        'tab size if inlist(rt_from_to, "A10TOU to B10", "E19 to B19")   , sort')

In [ ]:
#| export
def _is_cr_delimiter(delimiter):
    return delimiter in {'cr', None}

In [ ]:
#| export
delimit_regex = re.compile(r'#delimit(.*$)', flags=re.MULTILINE)
def _replace_delimiter(code, sc_delimiter=False):
    # Recursively replace custom delimiter with newline

    split = delimit_regex.split(code.strip(),maxsplit=1)

    if len(split) == 3:
        before = split[0]
        after = _replace_delimiter(split[2],not _is_cr_delimiter(split[1].strip()))
    else:
        before = code
        after = ''

    if sc_delimiter:
        before = before.replace('\r', ' ').replace('\n', ' ')
        before = before.replace(';','\n')

    return before + after

In [ ]:
show_doc(_replace_delimiter)

---

[source](https://github.com/hugetim/nbstata/blob/main/nbstata/code_utils.py#L23){target="_blank" style="float:right; font-size:smaller"}

### _replace_delimiter

>      _replace_delimiter (code, sc_delimiter=False)

Note: the following more-complicated regex would detect valid delimiters plus macros: 
```python
delimit_regex = re.compile(r'#delimit( |\t)+(;|cr|`.+\'|\$_.+|\$.+)')
```
but that's unnecessary, since Stata's `#delimit x` interprets any `x` other than 'cr' as switching the delimiter to ';'.

In [ ]:
test_eq(
    _replace_delimiter(dedent("""\
        list var1
        #delimit;
        list var2;list var3;
        list
        var4;
        """)),
    dedent("""\
        list var1
         list var2
        list var3
         list var4
        """)
)

In [ ]:
test_eq(_replace_delimiter(dedent("""\
    disp "start"
    #delimit;
    disp "hello"; disp "hello2";
    disp 
        "hello2a";
    #delimit cr
    disp "hello3"
    disp "hello4"
    #delimit;""")), 
        dedent("""\
    disp "start"
    disp "hello"
     disp "hello2"
     disp      "hello2a"
     disp "hello3"
    disp "hello4"
    """))

In [ ]:
#| export
def ending_sc_delimiter(code, sc_delimiter=False):
    code = _remove_multi_line_comments(code)
    # Recursively determine ending delimiter
    split = delimit_regex.split(code.strip(),maxsplit=1)
    if len(split) == 3:
        sc_delimiter = ending_sc_delimiter(split[2], not _is_cr_delimiter(split[1].strip()))
    elif len(split) == 2:
        sc_delimiter = not _is_cr_delimiter(split[1].strip())
    return sc_delimiter

In [ ]:
test_eq(
    ending_sc_delimiter(dedent("""\
        list var1
        #delimit;
        list var2;list var3;
        """)),
    True)

In [ ]:
test_eq(
    ending_sc_delimiter(dedent("""\
        /*
        #delimit;
        */
        disp 1
        disp 2""")),
    False)

In [ ]:
#| hide
test_eq(
    ending_sc_delimiter(dedent("""\
        #delimit;
        scalar
        list x""")),
    True)

In [ ]:
#| export

# Detect Multiple whitespace
multi_regex = re.compile(r' +')

def standardize_code(code, sc_delimiter=False):
    """Remove comments spanning multiple lines and replace custom delimiters"""
    code = _remove_multi_line_comments(code)
    
    # After removing multi-line comments, which could include "#delimit;"
    code = _replace_delimiter(code, sc_delimiter) 
    
    # Replace multiple whitespace with one
    code = multi_regex.sub(' ',code)
    
    # Delete blank lines and whitespace at start and end of lines
    code_lines = code.splitlines()
    std_lines = []
    for code_line in code_lines:
        cs = code_line.strip()
        if cs:
            std_lines.append(cs)
    return '\n'.join(std_lines)

In [ ]:
test_eq(
    standardize_code(dedent("""\
        list var1
        #delimit;
        list var2; list var3;
        list
        var4;
        """)), 
    dedent("""\
        list var1
        list var2
        list var3
        list var4""")
)

In [ ]:
test_eq(
    standardize_code(dedent("""\
        /*
        blah
        blah
        */
        list var
        """)), 
    "list var")

In [ ]:
#| hide
test_eq(
    standardize_code(dedent("""\
        /*
        #delimit;
        */
        disp 1
        disp 2
        """)), 
    dedent("""\
        disp 1
        disp 2""")
)

In [ ]:
#| hide
test_eq(
    standardize_code(dedent("""\
        disp ///
        1
        """)),
    "disp 1")

In [ ]:
test_eq(
    standardize_code(dedent("""\
        disp /// comment
        1
        """)),
    "disp 1")

In [ ]:
#| hide
test_eq(
    standardize_code(dedent("""\
        list var
        """)), 
    "list var")

In [ ]:
test_eq(standardize_code("list    var"), "list var")

In [ ]:
standardize_code('''\
display "displayed1"
/*
display "displayed2"
*/
display "displayed3"''')

'display "displayed1"\ndisplay "displayed3"'

In [ ]:
#| hide
standardize_code('''\
display "line continuation " /// commented out
    "comment"''')

'display "line continuation " "comment"'

## Separate out Stata program code
...because [such code](https://www.stata.com/manuals/pprogram.pdf) (as well as [python](https://www.stata.com/stata-news/news35-3/python-blogs/)/[mata](https://www.stata.com/manuals/m-1first.pdf) blocks) is unsuitable for `run_as_program`

In [ ]:
#| export
def _startswith_stata_abbrev(string, full_command, shortest_abbrev):
    for j in range(len(shortest_abbrev), len(full_command)+1):
        if string.startswith(full_command[0:j] + ' '):
            return True
    return False

In [ ]:
#| hide
test_eq(_startswith_stata_abbrev("q list var", "quietly", "q"), True)
test_eq(_startswith_stata_abbrev("qui list var", "quietly", "q"), True)

In [ ]:
#| export
def _remove_prog_prefixes(cs):
    if (_startswith_stata_abbrev(cs, 'quietly', 'qui')
        or cs.startswith('capture ')
        or _startswith_stata_abbrev(cs, 'noisily', 'n')):
        return _remove_prog_prefixes(cs.split(None, maxsplit=1)[1])
    else:
        return cs

In [ ]:
#| hide
test_eq(_remove_prog_prefixes("capture noisily program test_program"), "program test_program")

In [ ]:
#| export
def is_start_of_program_block(std_code_line):
    cs = _remove_prog_prefixes(std_code_line)
    _starts_program = (_startswith_stata_abbrev(cs, 'program', 'pr')
                       and not (cs.split()[1] in ['di', 'dir', 'drop', 'l', 'li', 'lis', 'list']))
    return (_starts_program
            or (cs in {'mata', 'mata:'})
            or (cs in {'python', 'python:'}))

In [ ]:
test_eq(is_start_of_program_block("capture noisily program test_program"), True)
test_eq(is_start_of_program_block("capture noisily list var"), False)
test_eq(is_start_of_program_block("pr l display1"), False)

In [ ]:
#| export
def _prog_blocks(std_code_lines):
    next_block_lines = []
    in_program = False
    for std_code_line in std_code_lines:         
        if is_start_of_program_block(std_code_line):
            if next_block_lines: # previous lines
                yield _block(next_block_lines, is_prog=in_program)
                next_block_lines = []
            in_program = True
        next_block_lines.append(std_code_line)
        if std_code_line == 'end': # regardless of whether in_program
            yield _block(next_block_lines, is_prog=True)
            next_block_lines = []
            in_program = False
    if next_block_lines:
        yield _block(next_block_lines, in_program)
        

def _block(block_lines, is_prog):
    return {"is_prog": is_prog, "std_code": '\n'.join(block_lines)}

In [ ]:
#| export
def break_out_prog_blocks(code, sc_delimiter=False):
    std_code_lines = standardize_code(code, sc_delimiter).splitlines()
    return list(_prog_blocks(std_code_lines))

In [ ]:
test_eq(
    break_out_prog_blocks(dedent('''\
        capture program drop ender
        program define ender
            disp "ender output"
        end
        capture program drop display2
        program define display2
            ender
        end
        display2
        ''')),
    [{'is_prog': False, 'std_code': 'capture program drop ender'},
     {'is_prog': True,
      'std_code': 'program define ender\ndisp "ender output"\nend'},
     {'is_prog': False, 'std_code': 'capture program drop display2'},
     {'is_prog': True, 'std_code': 'program define display2\nender\nend'},
     {'is_prog': False, 'std_code': 'display2'}]
)

In [ ]:
#| hide
test_eq(
    break_out_prog_blocks(dedent('''\
        program define ender
            disp "ender output"
        ''')),
    [{'is_prog': True,
      'std_code': 'program define ender\ndisp "ender output"'}]
)

In [ ]:
#| hide
break_out_prog_blocks('''\
display "line continuation " /// commented out
    "comment"''')

[{'is_prog': False, 'std_code': 'display "line continuation " "comment"'}]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()